<a href="https://colab.research.google.com/github/shabalin13/code-search/blob/main/delivery3/PML%26DL_delivery3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/shabalin13/code-search/blob/code-cleaning-delivery3/delivery3/PML%26DL_delivery2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Delivery 2

##Implementing search

In [7]:
EMBEDDINGS_PRECOMPUTED = True
EMBEDDINGS_ON_GOOGLE_DRIVE = True
if EMBEDDINGS_ON_GOOGLE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers --quiet
!pip install datasets --quiet
!apt install libomp-dev
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from enum import Enum, auto
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    !pip install faiss-gpu -quiet
else:
    !pip install faiss -quiet

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

dataset = load_dataset("code_x_glue_ct_code_to_text", 'python')

     |████████████████████████████████| 5.5 MB 28.0 MB/s 
     |████████████████████████████████| 163 kB 42.8 MB/s 
     |████████████████████████████████| 7.6 MB 22.3 MB/s 
     |████████████████████████████████| 441 kB 24.2 MB/s 
     |████████████████████████████████| 212 kB 74.0 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |████████████████████████████████| 115 kB 64.4 MB/s 
     |████████████████████████████████| 127 kB 46.0 MB/s 
     |████████████████████████████████| 115 kB 67.9 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 5 not upgraded.
Need to get 239 kB of a

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

Dataset code_x_glue_ct_code_to_text downloaded and prepared to /root/.cache/huggingface/datasets/code_x_glue_ct_code_to_text/python/0.0.0/f8b7e9d51f609a87e7ec7c7431706d4ee0b402e3398560410313d4acc67060a0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train, validatation, test = dataset['train'], dataset['validation'], dataset['test']

In [4]:
class SeqType(Enum):
  CODE = auto()
  DOC = auto()


class TokenizeCollator(object):
    def __init__(self, tokenizer, seq_type):
        self.tokenizer = tokenizer
        self.seq_type = seq_type

    def __call__(self, batch):
        return self.create_one_batch(batch)

    def create_one_batch(self, batch):
      tokens_batch = list(map(lambda item: self.get_formatted_input(item), batch))
      batch_encoding = self.tokenizer(tokens_batch, padding=True, return_tensors='pt', return_token_type_ids=True, truncation=True)
      tokens_ids = batch_encoding.input_ids.to(device)
      token_type_ids = batch_encoding.token_type_ids.to(device)
      attention_mask = batch_encoding.attention_mask.to(device)
      return tokens_ids, token_type_ids, attention_mask

    def get_formatted_input(self, item):
      if self.seq_type == SeqType.CODE:
        return self.get_formatted_input_for_code(item)
      elif self.seq_type == SeqType.DOC:
        return self.get_formatted_input_for_doc(item)
      else:
        raise Exception("Incorrect sequence type")

    def get_formatted_input_for_code(self, item):
      doc_tokens = ' '.join(item['docstring_tokens'])
      code_tokens = ' '.join(item['code_tokens'])
      formatted_input = self.tokenizer.cls_token + doc_tokens + self.tokenizer.sep_token+code_tokens + self.tokenizer.sep_token
      return formatted_input

    def get_formatted_input_for_doc(self, item):
      doc_tokens = ' '.join(item['docstring_tokens'])
      code_tokens = ''
      formatted_input = self.tokenizer.cls_token + doc_tokens + self.tokenizer.sep_token+code_tokens + self.tokenizer.sep_token
      return formatted_input 


code_tokenize_collate_fn = TokenizeCollator(tokenizer, SeqType.CODE)
doc_tokenize_collate_fn = TokenizeCollator(tokenizer, SeqType.DOC)

BATCH_SIZE = 256
test_code_tokens_ids = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=code_tokenize_collate_fn, num_workers=0)
test_doc_tokens_ids = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=doc_tokenize_collate_fn, num_workers=0)

# for idx, batch in enumerate(test_tokens_ids):
#   # print(batch.shape)
#   print(batch)
#   if idx >= 0:
#     break

In [5]:
# torch.cuda.empty_cache()
model.to(device)


def print_gpu_memory_usage(idx=''):
  print(idx)
  print('Memory Usage:')
  print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
  print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
  print()


if not EMBEDDINGS_PRECOMPUTED:
  batched_test_code_embs = []
  for batch in tqdm(test_code_tokens_ids):
    tokens_ids, token_type_ids, attention_mask = batch
    with torch.no_grad():
      embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
      batched_test_code_embs.append(embs)

  batched_test_doc_embs = []
  for batch in tqdm(test_doc_tokens_ids):
    tokens_ids, token_type_ids, attention_mask = batch
    with torch.no_grad():
      embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
      batched_test_doc_embs.append(embs)

  test_code_embeddings = torch.cat(batched_test_code_embs, dim=0)
  test_doc_embeddings = torch.cat(batched_test_doc_embs, dim=0)
  if EMBEDDINGS_ON_GOOGLE_DRIVE:
    %cd /content/drive/MyDrive/PML&DL/Project
  torch.save(test_code_embeddings, 'test_code_embeddings.pt')
  torch.save(test_doc_embeddings, 'test_doc_embeddings.pt')

In [8]:
if EMBEDDINGS_PRECOMPUTED:
  if EMBEDDINGS_ON_GOOGLE_DRIVE:
    %cd /content/drive/MyDrive/PML&DL/Project
  test_code_embeddings = torch.load('test_code_embeddings.pt', map_location=torch.device('cpu'))
  test_doc_embeddings = torch.load('test_code_embeddings.pt', map_location=torch.device('cpu'))

/content/drive/MyDrive/PML&DL/Project


In [9]:
import numpy as np
import faiss

class FaissKNeighbors:
    def __init__(self, is_cuda):
        self.index = None
        self.is_cuda = is_cuda

    def fit(self, X):
        self.index = faiss.IndexFlatL2(X.shape[1])
        if self.is_cuda:
          res = faiss.StandardGpuResources()
          self.index = faiss.index_cpu_to_gpu(res, 0, self.index)
        if type(X) == torch.Tensor:
          X = X.numpy()
        self.index.add(X)

    def predict(self, X, k):
        if type(X) == torch.Tensor:
          X = X.numpy()
        distances, indices = self.index.search(X, k=k)
        return indices

ModuleNotFoundError: ignored

In [ ]:
test_faiss = FaissKNeighbors(is_cuda=device=='cuda')

In [ ]:
k = 1000
mrrs = []
for beg_idx in tqdm(range(0, len(test_code_embeddings), k)):
  if beg_idx + k > len(test_code_embeddings):
    break
  doc_embs_subset = test_doc_embeddings[beg_idx:beg_idx + k]
  code_embs_subset = test_code_embeddings[beg_idx:beg_idx + k]
  test_faiss.fit(code_embs_subset)  
  preds = test_faiss.predict(doc_embs_subset, k=k)

  targets = np.repeat(np.expand_dims(range(k), 1), k, axis=1)

  reciprocal_ranks = 1 / (np.argwhere(np.equal(preds, targets))[:,1] + 1)
  mrr_ = np.mean(reciprocal_ranks)
  mrrs.append(mrr_)

In [ ]:
mrr = np.mean(mrrs)
print('Mean Reciprocal rank is: ', mrr)